<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Дисбаланс" data-toc-modified-id="Дисбаланс-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Дисбаланс</a></span></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Линейная-модель" data-toc-modified-id="Линейная-модель-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Линейная модель</a></span></li><li><span><a href="#Дерево" data-toc-modified-id="Дерево-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Дерево</a></span></li><li><span><a href="#Лес" data-toc-modified-id="Лес-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Лес</a></span></li><li><span><a href="#DistilBert" data-toc-modified-id="DistilBert-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>DistilBert</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» 

*F1* не меньше 0.75. 

## Подготовка

In [1]:
import pandas as pd
import nltk
from tqdm import tqdm
tqdm.pandas()
import re 
import scipy
from scipy import sparse

import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline

from scipy.sparse import csr_matrix

from nltk.corpus import stopwords as nltk_stopwords

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
stopwords = list(nltk_stopwords.words('english'))

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [ ]:
df.info()

Данные без пропусков

In [3]:
features = df['text']
target = df['toxic']

In [4]:
features_train, features_test, target_train, target_test =\
train_test_split(features, target, test_size=0.33, random_state=42)

### Дисбаланс

In [ ]:
plot = df.groupby('toxic')['toxic'].count().plot.pie()

В данных явный дисбаланс, который отразится на качестве моделей. Избавимся от этой проблемы

In [5]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

# features_downsampled, target_downsampled = downsample(features_train, target_train, 0.12)

In [ ]:
# plot = target_downsampled.groupby(target_downsampled).count().plot.pie()

In [21]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

# features_upsampled, target_upsampled = upsample(features_train, target_train, 8)

In [ ]:
# plot = target_upsampled.groupby(target_upsampled).count().plot.pie()

### Подготовка

In [6]:
def lemmatize(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text) 
    text = ' '.join(text.split())
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    lemm_text = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemm_text

In [7]:
features_train = features_train.progress_apply(lemmatize)

100%|██████████| 106725/106725 [01:04<00:00, 1647.02it/s]


In [8]:
features_test = features_test.progress_apply(lemmatize)

100%|██████████| 52567/52567 [00:31<00:00, 1677.56it/s]


In [ ]:
# features_downsampled = features_downsampled.progress_apply(lemmatize)

In [ ]:
# features_upsampled = features_upsampled.progress_apply(lemmatize)

In [9]:
%%time 

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train_tf = count_tf_idf.fit_transform(features_train)
features_test_tf = count_tf_idf.transform(features_test)

CPU times: user 7.41 s, sys: 124 ms, total: 7.54 s
Wall time: 7.54 s


## Обучение

Чтобы не дублироваться, запишем функцию обучения и проверки модели:

In [ ]:
def f1_cross(model, features, target):
    model = model
    model.fit(features, target)
    f1 = cross_val_score(model,
                    features, 
                    target, 
                    cv = 3, 
                    n_jobs = -1, 
                    scoring = 'f1').mean()
    return round(f1, 2)

In [ ]:
results = pd.DataFrame({
    'Model' : [],
    'f1' : []
})

### Линейная модель

Сравним качество линейной модели для обычного пайплайна и  imblearn:

In [10]:
pipe = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('lin_m', LogisticRegression())
])

In [11]:
param_grid = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'lin_m__C' : [1,2,6]
}

In [12]:
%%time

grid_search_tune = RandomizedSearchCV(
    pipe,
    param_grid, 
    cv=3,
    n_jobs=-1,
    scoring='f1',
    verbose=10)

grid_search_tune.fit(features_train, target_train)

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3; 1/9] START lin_m__C=1, tfidf__max_df=0.25..............................
[CV 1/3; 1/9] END ............lin_m__C=1, tfidf__max_df=0.25; total time=  38.9s
[CV 2/3; 1/9] START lin_m__C=1, tfidf__max_df=0.25..............................
[CV 2/3; 1/9] END ............lin_m__C=1, tfidf__max_df=0.25; total time=  40.4s
[CV 3/3; 1/9] START lin_m__C=1, tfidf__max_df=0.25..............................


KeyboardInterrupt: 

In [ ]:
grid_search_tune.best_params_

{'tfidf__max_df': 0.25, 'lin_m__C': 6}

In [ ]:
grid_search_tune.best_score_

<div class="alert alert-success">
<b>Комментарий ревьюера V2✔️:</b> Да, отлично, получилось хорошо
</div>

0.7592110337653272

In [ ]:
results.loc[len(results.index)] = ['LR_regular_pipe', grid_search_tune.best_score_]

In [ ]:
# pip install imblearn

In [9]:
from imblearn.pipeline import Pipeline as imbPipeline
from imblearn.over_sampling import RandomOverSampler

**Если будет ругаться, нужно запустить pip install, а потом сбросить ядро**

In [10]:
pipe = imbPipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('oversampling', RandomOverSampler(random_state=0)),
    ('lin_m', LogisticRegression())
])

In [11]:
param_grid = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'lin_m__C' : [1,2,6]
}

In [12]:
%%time

grid_search_tune = RandomizedSearchCV(
    pipe,
    param_grid, 
    cv=3,
    n_jobs=-1,
    scoring='f1',
    verbose=10)

grid_search_tune.fit(features_train, target_train)

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3; 1/9] START lin_m__C=1, tfidf__max_df=0.25..............................


KeyboardInterrupt: 

In [ ]:
grid_search_tune.best_params_

In [ ]:
grid_search_tune.best_score_

0.7618024562150874

**Едва перевалили порог**

In [ ]:
results.loc[len(results.index)] = ['LR_imb_pipe', grid_search_tune.best_score_]

Результат не сильно улучшился, если реуглировать только max_df

Лучше всего оказались данные с апсамплингом, с почти идеальным результатом

In [ ]:
# from sklear.base import BaseEstimator, TransformerMixin

# class DoSomething(BaseEstimator, TransformerMixin):
	
# 	def fit(self, X, y):
# 		return self

# 	def tramsform(self, X):
# 		return X.drop(['Name'], axis=1)

# # TransformerMixin сразу же добавляет fit_transform

### Дерево

In [13]:
pipe = imbPipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords, max_df=0.25)),
    ('oversampling', RandomOverSampler(random_state=0)),
    ('tree', DecisionTreeClassifier(random_state=0))
])

In [14]:
param_grid = {
    'tree__max_depth': range(3, 11, 2),
    'tree__min_samples_leaf': range(4, 8),
    'tree__min_samples_split': range(2, 6, 2)
             }

In [15]:
%%time

grid_search_tune = RandomizedSearchCV(
    pipe,
    param_grid, 
    cv=3,
    n_jobs=-1,
    scoring='f1',
    verbose=10)

grid_search_tune.fit(features_train, target_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START tree__max_depth=7, tree__min_samples_leaf=4, tree__min_samples_split=4


KeyboardInterrupt: 

In [ ]:
grid_search_tune.best_params_

{'tree__min_samples_split': 4,
 'tree__min_samples_leaf': 5,
 'tree__max_depth': 9}

In [ ]:
 grid_search_tune.best_score_

0.5529802580453375

In [ ]:
results.loc[len(results.index)] = ['tree', grid_search_tune.best_score_]

In [ ]:
# %%time
# model_tree = DecisionTreeClassifier(random_state=0)
# grid = (model_tree,
#                        param_grid=parametrs,
#                        scoring='f1',
#                        n_jobs=-1,
#                        cv=3,
#                        verbose=10)
# grid.fit(features_train, target_train)
# grid.best_params_

{'max_depth': 9, 'min_samples_leaf': 7, 'min_samples_split': 2}

Очень плохой результат на фоне линейной модели

### Лес

Лес окажется лучше всех, и его нужно будет протестировать. Чтобы заново не обучать модель, сделаем это отдельно, без функции

In [16]:
pipe = imbPipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('oversampling', RandomOverSampler(random_state=0)),
    ('forest', RandomForestClassifier(random_state=0))
])

In [17]:
f1 = cross_val_score(pipe,
                features_train, 
                target_train, 
                cv = 3, 
                n_jobs = -1, 
                scoring = 'f1').mean()
f1 = round(f1, 2)
results.loc[len(results.index)] = ['Forest', f1]

KeyboardInterrupt: 

In [ ]:
results

### DistilBert

Подумала, что легкая модель может заработать, но ядро все равно умирает

In [ ]:
# import numpy as np
# import pandas as pd
# import torch
# from torch.nn.utils.rnn import pad_sequence
# from tqdm import tqdm
# from tqdm import notebook
# tqdm.pandas()

# import transformers as ppb
# from transformers import DistilBertConfig, DistilBertModel
# from transformers import BertTokenizer
# from transformers.models.bert.modeling_bert import BertModel

In [ ]:
# model_class, tokenizer_class, pretrained_weights = (
#     ppb.DistilBertModel,
#     ppb.DistilBertTokenizer,
#     'distilbert-base-uncased')

# tokenizer = BertTokenizer.from_pretrained("unitary/toxic-bert")
# model = BertModel.from_pretrained("unitary/toxic-bert")

In [ ]:
# %%time

# tokenized = df['text'].progress_apply(
#     lambda x: tokenizer.encode(x, max_length=512, truncation=True, add_special_tokens=True))

In [ ]:
# padded = pad_sequence([torch.as_tensor(seq) for seq in tokenized], batch_first=True)

In [ ]:
# attention_mask = padded > 0
# attention_mask = attention_mask.type(torch.LongTensor)

In [ ]:
# %%time
# input_ids = torch.tensor(np.array(padded))

# with torch.no_grad():
#     last_hidden_states = model(input_ids) 

In [ ]:
# %%time
# input_ids = torch.tensor(np.array(tokenized_test))

# with torch.no_grad():
#     last_hidden_states_test = model(input_ids)  

In [ ]:
# %%time

# batch_size = 10 

# embeddings = []

# for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
#     batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
#     attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]) 
    
#     with torch.no_grad():
#         batch_embeddings = model(batch, attention_mask=attention_mask_batch)  
        
#     embeddings.append(batch_embeddings[0][:,0,:].numpy()) 
    
# features = np.concatenate(embeddings)

In [ ]:
# bert_train, bert_test, target_train, target_test =\
# train_test_split(tokenized, target, test_size=0.33, random_state=42)

In [ ]:
# bert_train = last_hidden_states[0][:,0,:].numpy()
# bert_test = last_hidden_states_test[0][:,0,:].numpy()

In [ ]:
# f1_bert = f1_cross(LogisticRegression(), bert_train, target_upsampled)

## Выводы

In [ ]:
results

Лучше всего оказались линейная модель и лес деревьев.

In [18]:
model = LogisticRegression(C=6)

In [19]:
%%time
model.fit(features_train_tf, target_train)

CPU times: user 22.6 s, sys: 25.2 s, total: 47.8 s
Wall time: 47.9 s


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=6)

In [20]:
f1_score(target_test, model.predict(features_test_tf))

0.7634028892455859